# Collaborative Filtering Recommendations using Original Values

In [104]:
import pandas as pd
import numpy as np
import implicit

from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix

In [105]:
train = pd.read_csv("data\\train-plays.csv")
test = pd.read_csv("data\\test-plays.csv")
game_coding = pd.read_csv("data\\game-coding.csv")

In [106]:
train.shape[0]

36447

## Format training data

In [107]:
game_user = train.pivot(index="game_id", columns="user_id", values="amount")

In [108]:
game_user = game_user.fillna(0)

In [109]:
user_coding = pd.DataFrame({"original":game_user.columns, "coded":np.arange(game_user.columns.size)})

In [110]:
game_user_sparse = csr_matrix(game_user)

In [111]:
user_game = game_user.T

In [112]:
user_game_sparse = csr_matrix(user_game)

In [113]:
user_ids = train['user_id'].unique()

In [114]:
matrix_size = game_user_sparse.shape[0] * game_user_sparse.shape[1] # Number of possible interactions in the matrix
num_played = len(game_user_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100 * (1 - (num_played / matrix_size))
sparsity

96.98755248454393

## Alternating Least Squares

In [143]:
als = implicit.als.AlternatingLeastSquares(128, 0.05, iterations=50)

In [137]:
als.fit(game_user_sparse)

In [138]:
game_recs = np.ndarray.flatten(als.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})

In [139]:
results = recommendations.merge(test, on="user_id")

In [140]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [141]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.1810344827586207

In [142]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.7334611697027804

# Bayesian Personalized Ranking

In [53]:
bpr = implicit.bpr.BayesianPersonalizedRanking(128, 0.01, 0.05, iterations=200)

In [54]:
bpr.fit(game_user_sparse)

In [55]:
game_recs = np.ndarray.flatten(bpr.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})

In [56]:
results = recommendations.merge(test, on="user_id")

In [57]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [58]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.19482758620689655

In [59]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.9731543624161074

# Logistic Matrix Factorization

In [173]:
lmf = implicit.lmf.LogisticMatrixFactorization(128)

In [167]:
lmf.fit(game_user_sparse)
print(lmf.iterations, lmf.regularization, lmf.learning_rate, lmf.neg_prop)


30 2.0 0.85 30


In [168]:
game_recs = np.ndarray.flatten(lmf.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})

In [169]:
results = recommendations.merge(test, on="user_id")

In [170]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [171]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.056896551724137934

In [172]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.009587727708533078

# ALS + BPR

In [87]:
user_recs = dict()
for user_id in user_ids:
    res = set(idx for idx, score in als.recommend(user_id, user_game_sparse))
    user_recs[user_id] = res

In [88]:
for user_id in user_ids:
    res = set(idx for idx, score in bpr.recommend(user_id, user_game_sparse))
    als_res = user_recs[user_id]
    for r in res:
        als_res.add(r)
    user_recs[user_id] = als_res

In [89]:
num_total = 0
num_correct = 0
for user_id in user_ids:
    test_df = test[test['user_id'] == user_id]
    test_game_id = test_df['game_id'][user_id]
    num_total = num_total + 1
    if test_game_id in user_recs[user_id]:
        num_correct = num_correct + 1
        
print(num_correct / num_total)

0.2810344827586207


# Average Accuracy Calculation

In [ ]:
accuracy = 0
seen = 0
for i in range(0, 100):
    als.fit(game_user_sparse)
    game_recs = np.ndarray.flatten(als.recommend_all(user_game_sparse))
    user_10 = np.repeat(user_game.index, 10)
    recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})
    results = recommendations.merge(test, on="user_id")
    results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)
    # accuracy
    accuracy = accuracy + (results[results.recommended == 1].shape[0] / test.shape[0])
    # % of games recommended
    seen = seen + (np.unique(game_recs).size / train.game_id.unique().size)

In [ ]:
accuracy / 100

In [ ]:
seen / 100

In [147]:
accuracy = 0
seen = 0
for i in range(0, 100):
    bpr.fit(game_user_sparse)
    game_recs = np.ndarray.flatten(bpr.recommend_all(user_game_sparse))
    user_10 = np.repeat(user_game.index, 10)
    recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})
    results = recommendations.merge(test, on="user_id")
    results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)
    # accuracy
    accuracy = accuracy + (results[results.recommended == 1].shape[0] / test.shape[0])
    # % of games recommended
    seen = seen + (np.unique(game_recs).size / train.game_id.unique().size)

In [148]:
accuracy / 100

0.19918103448275865

In [149]:
seen / 100

0.9731351869606906

In [174]:
accuracy = 0
seen = 0
for i in range(0, 100):
    lmf.fit(game_user_sparse)
    game_recs = np.ndarray.flatten(lmf.recommend_all(user_game_sparse))
    user_10 = np.repeat(user_game.index, 10)
    recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})
    results = recommendations.merge(test, on="user_id")
    results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)
    # accuracy
    accuracy = accuracy + (results[results.recommended == 1].shape[0] / test.shape[0])
    # % of games recommended
    seen = seen + (np.unique(game_recs).size / train.game_id.unique().size)

In [175]:
accuracy / 100

0.05689655172413786

In [176]:
seen / 100

0.009587727708533089